In [ ]:
from typing import Tuple
import numpy as np

def generate_grid_mesh(height: int, width: int) -> Tuple[np.ndarray, np.ndarray]:
  # define grid offsets
  grid_offsets = np.zeros((height + 1, width + 1, 2), dtype=np.int32)
  grid_offsets[:, :, 0] = np.arange(height + 1)[:, None]
  grid_offsets[:, :, 1] = np.arange(width + 1)[None, :]
  
  unique_grid_unit_points = np.array([[0.0, 0.0], [0.5, 0.0], [0.0, 0.5], [0.5, 0.5]])

  # h+1 x w+1 x 4 x 2
  grid_points = unique_grid_unit_points[None, None, :] + np.flip(grid_offsets, -1)[:, :, None, :]

  grid_indices = np.zeros((height + 1, width + 1, 4, 3), dtype=np.int32)
  grid_indices[:, :, :, 0] = np.arange(4)[None, None, :]
  grid_indices[:, :, :, 1:] = grid_offsets[:, :, None, :]

  # h x w x 4 x 3
  a = grid_indices[:-1, :-1]
  b = grid_indices[:-1, 1:]
  c = grid_indices[1:, :-1]
  d = grid_indices[1:, 1:]
  gridded_triangles = [
    (a[:, :, 0], a[:, :, 3], a[:, :, 1]),
    (a[:, :, 0], a[:, :, 2], a[:, :, 3]),
    (a[:, :, 1], a[:, :, 3], b[:, :, 0]),
    (a[:, :, 3], b[:, :, 2], b[:, :, 0]),
    (a[:, :, 2], c[:, :, 0], a[:, :, 3]),
    (a[:, :, 3], c[:, :, 0], c[:, :, 1]),
    (a[:, :, 3], c[:, :, 1], d[:, :, 0]),
    (a[:, :, 3], d[:, :, 0], b[:, :, 2])
  ]
  triangles = []
  for pointset in gridded_triangles:
    # h x w x 3 x 3 (third axis is the grid coordinate)
    pointset = np.stack(pointset, axis=-1)
    # h x w x 3 (grid coordinate converted to flat index)
    pointset = (
      pointset[:, :, 2, :] * 4
      + pointset[:, :, 1, :] * (width + 1) * 4
      + pointset[:, :, 0, :]
    )
    # (h x w) x 3 (grid of flat index triplets converted to list of triplets)
    pointset = pointset.reshape((-1, 3))
    triangles.append(pointset)
  # (8 x h x w) x 3 (list of triangles - 8 per grid face)
  triangles = np.concatenate(triangles)

  points = grid_points.reshape((-1, 2))
  points = np.concatenate((points, np.zeros((points.shape[0], 1))), axis=-1)
  return points, triangles

points, triangles = generate_grid_mesh(4, 2)
save_stl(points, triangles, 'output.stl')

gp shape (5, 3, 4, 2)
a [[[[0 0 0]
   [1 0 0]
   [2 0 0]
   [3 0 0]]

  [[0 0 1]
   [1 0 1]
   [2 0 1]
   [3 0 1]]]


 [[[0 1 0]
   [1 1 0]
   [2 1 0]
   [3 1 0]]

  [[0 1 1]
   [1 1 1]
   [2 1 1]
   [3 1 1]]]


 [[[0 2 0]
   [1 2 0]
   [2 2 0]
   [3 2 0]]

  [[0 2 1]
   [1 2 1]
   [2 2 1]
   [3 2 1]]]


 [[[0 3 0]
   [1 3 0]
   [2 3 0]
   [3 3 0]]

  [[0 3 1]
   [1 3 1]
   [2 3 1]
   [3 3 1]]]]
c [[[[0 1 0]
   [1 1 0]
   [2 1 0]
   [3 1 0]]

  [[0 1 1]
   [1 1 1]
   [2 1 1]
   [3 1 1]]]


 [[[0 2 0]
   [1 2 0]
   [2 2 0]
   [3 2 0]]

  [[0 2 1]
   [1 2 1]
   [2 2 1]
   [3 2 1]]]


 [[[0 3 0]
   [1 3 0]
   [2 3 0]
   [3 3 0]]

  [[0 3 1]
   [1 3 1]
   [2 3 1]
   [3 3 1]]]


 [[[0 4 0]
   [1 4 0]
   [2 4 0]
   [3 4 0]]

  [[0 4 1]
   [1 4 1]
   [2 4 1]
   [3 4 1]]]]
triangles [array([[ 0,  3,  1],
       [ 4,  7,  5],
       [12, 15, 13],
       [16, 19, 17],
       [24, 27, 25],
       [28, 31, 29],
       [36, 39, 37],
       [40, 43, 41]], dtype=int32), array([[ 0,  2,  3],
       [ 

In [ ]:
from stl import mesh

def save_stl(vertices, faces, filename):
    """
    Save vertex and face data as an STL file.

    @param vertices: n x 3 array of 3D points.
    @param faces: m x 3 array of vertex indices specifying triangles.
    """
    # Create the mesh
    triangle_mesh = mesh.Mesh(np.zeros(faces.shape[0], dtype=mesh.Mesh.dtype))
    
    # Fill in the mesh vertices for each triangle
    for i, face in enumerate(faces):
        for j in range(3):
            triangle_mesh.vectors[i][j] = vertices[face[j]]
    
    # Save to file
    triangle_mesh.save(filename)

# Example usage
vertices = np.array([
    [0, 0, 0],
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1]
])

faces = np.array([
    [0, 1, 2],
    [0, 1, 3],
    [0, 2, 3],
    [1, 2, 3]
])

save_stl(points, triangles, 'output.stl')